In [64]:
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import timeit
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Conv2D, CuDNNGRU, CuDNNLSTM, \
    TimeDistributed, Reshape, Input, Dropout, RepeatVector, MaxPooling2D, \
    UpSampling2D, Cropping2D
from keras.activations import relu
from keras.preprocessing.image import ImageDataGenerator

In [65]:
T_pred = 16
T_in = 16
vid_l = T_in + T_pred
IMG_H = 125
IMG_W = 125
IMG_CH = 3

In [66]:
TD = TimeDistributed
conv = lambda x: Conv2D(filters=x, kernel_size=(5,5), strides=(2, 2), padding='valid',
                        data_format=None, dilation_rate=(1, 1), use_bias=True,
                        kernel_initializer='glorot_uniform', bias_initializer='zeros')

inputs = Input(shape=(T_in, IMG_H, IMG_W, IMG_CH))

# Apply convolutions on the initial image input with increasing channel size.
print(inputs.shape)
conv_x = TD(conv(12))(inputs)
conv_x = Activation("relu")(conv_x)
print(conv_x.shape)
conv_x = TD(conv(24))(conv_x)
conv_x = Activation("relu")(conv_x)
print(conv_x.shape)
conv_x = TD(MaxPooling2D(pool_size=(2, 2)))(conv_x)
print(conv_x.shape)
conv_x = TD(conv(42))(conv_x)
conv_x = Activation("relu")(conv_x)
print(conv_x.shape)

# We pass the flattened convolution output into a CuDNN-optimized LSTM.
# Outputs are disregarded for training but form the "encoded" representation.
enc_x = Reshape((T_in, -1))(conv_x)
encoded_output, hidden, cell = CuDNNLSTM(512, return_state=True, return_sequences=False)(enc_x)

# Decode model output.
dec_raw_x = RepeatVector(T_pred)(encoded_output) # Input(shape=(None, 512,))
dec_x, _, _ = CuDNNLSTM(512, return_state=True, return_sequences=True)(dec_raw_x, initial_state=[hidden, cell])

dec_x = TD(Dropout(0.5))(dec_x)
dec_x = TD(Dense(2048))(dec_x)
dec_x = TD(Activation("relu"))(dec_x)

dec_x = TD(Dropout(0.5))(dec_x)
dec_x = TD(Dense(IMG_H * IMG_W * IMG_CH))(dec_x)
predictions = TD(Activation("relu"))(dec_x)
predictions = TD(Reshape((IMG_H, IMG_W, IMG_CH)))(predictions)


model = Model(inputs=inputs, outputs=predictions)

(?, 16, 125, 125, 3)
(?, 16, 61, 61, 12)
(?, 16, 29, 29, 24)
(?, 16, 14, 14, 24)
(?, 16, 5, 5, 42)


In [67]:
import h5py

SAVES_PATH = "/home/ubuntu/saves/"
BATCH = 32

with h5py.File('small_dataset.h5', 'r') as f:
    X = f['X'][:]
    Y = f['Y'][:]

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X, Y, batch_size=BATCH, epochs=10)

Epoch 1/10
512/512 [==============================] - 14s 28ms/step - loss: 4969.0971
Epoch 2/10
512/512 [==============================] - 9s 17ms/step - loss: 1033.9331
Epoch 3/10
512/512 [==============================] - 10s 19ms/step - loss: 726.4277
Epoch 4/10
512/512 [==============================] - 10s 20ms/step - loss: 684.1950
Epoch 5/10
512/512 [==============================] - 10s 19ms/step - loss: 675.5536
Epoch 6/10
512/512 [==============================] - 9s 18ms/step - loss: 670.4887
Epoch 7/10
512/512 [==============================] - 9s 17ms/step - loss: 670.4278
Epoch 8/10
512/512 [==============================] - 9s 17ms/step - loss: 672.9116
Epoch 9/10
512/512 [==============================] - 9s 18ms/step - loss: 672.6358
Epoch 10/10
512/512 [==============================] - 9s 17ms/step - loss: 672.5628


In [3]:
preds = model.predict(np.array(X[-64:]))
import matplotlib.pyplot as plt
import numpy as np
plt.imshow(np.array(preds[-1][0], dtype=np.uint8))
plt.show()

NameError: name 'model' is not defined

In [1]:
import h5py
SAVES_PATH = "/home/ubuntu/saves/"
BATCH = 64

with h5py.File('dataset.h5', 'r') as f:
    X = f['X'][0]/255
    Y = f['Y'][0]/255

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
X

NameError: name 'np' is not defined